In [1]:
SAMPLE_TEXT = """
2199943210
3987894921
9856789892
8767896789
9899965678
"""

In [2]:
from collections import Counter

In [3]:
def tokenize_line(line):
    return [int(point) for point in list(line)]


def parse_text(raw_text):
    return [tokenize_line(l) for l in raw_text.split("\n") if l]

def read_input():
    with open("input.txt", "rt") as f:
        return f.read()

In [4]:
class Board:
    def __init__(self, lines):
        self.lines = lines
        self.max_row_index = len(lines) - 1
        self.max_col_index = len(lines[0]) - 1

    def point_exists(self, row, col):
        return 0 <= row <= self. max_row_index and 0 <= col <= self.max_col_index

    def find_adjacent_points(self, row, col):
        up = (row - 1, col)
        down = (row + 1, col)
        left = (row, col - 1)
        right = (row, col + 1)
        result = [(r, c, self.lines[r][c]) for r, c in [up, down, left, right] if self.point_exists(r, c)]
        return result

    def swap(self, swap_fn):
        for r in range(len(self.lines)):
            for c in range(len(self.lines[0])):
                self.lines[r][c] = swap_fn(r, c, self.lines[r][c])

    def __str__(self):
        result = ""
        for row in self.lines:
            result += "".join(str(v) if v is not None else ' ' for v in row)
            result += "\n"
        return result


def find_low_points(lines):
    board = Board(lines)

    result = []
    for r in range(0, len(lines)):
        for c in range(0, len(lines[0])):
            value = lines[r][c]
            adjacent_points = [p[2] for p in board.find_adjacent_points(r, c)]
            if value < min(adjacent_points):
                result.append(value)
    return result

In [5]:
points = find_low_points(parse_text(SAMPLE_TEXT))
sum(p + 1 for p in points)

15

In [6]:
points = find_low_points(parse_text(read_input()))
sum(p + 1 for p in points)

532

In [20]:
def paint_basins(lines):
    board = Board(lines)
    # Since 9s can't be part of a basin, swap them to None. Otherwise use '.' to
    # indicate an unpainted cell
    board.swap(lambda r, c, x: '.' if x != 9 else None)

    def could_simplify_board():
        # Looks for adjacent cells with different values and swaps them all to be the same
        # Returns true if any changes are made, false when no further simplifications are possible
        for r in range(0, len(lines)):
            for c in range(0, len(lines[0])):
                value = board.lines[r][c]
                if value is None:
                    continue
                adjacent_points = [v[2] for v in board.find_adjacent_points(r, c) if v[2]]
                adjacent_points.append(value)
                distinct_values = list(set(adjacent_points))
                if len(distinct_values) > 1:
                    base = distinct_values[0]
                    other_values = distinct_values[1:]
                    board.swap(lambda r, c, x: base if x in other_values else x)
                    return True
        return False

    next_region = 0
    for r in range(0, len(lines)):
        for c in range(0, len(lines[0])):
            value = board.lines[r][c]
            if value is None:
                continue
            adjacent_points = [v[2] for v in board.find_adjacent_points(r, c) if v[2] and v[2] != '.']
            if not adjacent_points:
                board.lines[r][c] = next_region
                next_region += 1
            else:
                board.lines[r][c] = adjacent_points[0]

    # Ham-fisted approach - keep trying to reduce the number of regions on the
    # board until there's nothing left to reduce.
    while could_simplify_board():
        pass
    return board

def find_largest_basins(board: Board):
    all_values = []
    for row in board.lines:
        all_values.extend(r for r in row if r)
    result = 1
    for _, size in Counter(all_values).most_common(3):
        result *= size
    return result

In [24]:
def paint_basins_2(lines):
    """
    A faster implementation of basin painting. When a blank tile (a '.') is found,
    it will mark that square and walk through all adjacent blank tiles to mark
    them with the same value.
    """
    board = Board(lines)

    def swap_fn(row, column, value):
        return '.' if value != 9 else None

    def paint_from(row, column, value):
        board.lines[row][column] = value
        for r, c, v in board.find_adjacent_points(row, column):
            if v != '.':
                continue
            paint_from(r, c, value)

    board.swap(swap_fn)
    next_colour = 0
    for row in range(len(board.lines)):
        for col in range(len(board.lines[0])):
            if board.lines[row][col] == '.':
                paint_from(row, col, next_colour)
                next_colour += 1
    return board

In [16]:
basins = paint_basins(parse_text(SAMPLE_TEXT))
find_largest_basins(basins)

1134

In [17]:
basins = paint_basins(parse_text(read_input()))
find_largest_basins(basins)

1110780

In [22]:
# How much faster is paint_basins_2?
%timeit paint_basins(parse_text(SAMPLE_TEXT))

884 µs ± 184 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
%timeit paint_basins_2(parse_text(SAMPLE_TEXT))

241 µs ± 89.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
